In [2]:
import urllib.request
import pandas as pd
import json
import numpy as np
import os
from IPython.display import display
import seaborn

# Define Functions Here

Ideally, we should be able to store the functions in a separate file but how you do that in Python using Jupyter Notebooks...no one seems to know for certain.

In [3]:
def downloadJson( theUrl ):
    response = urllib.request.urlopen(theUrl)
    data = response.read()
    text = data.decode('utf-8')
    jsonData = json.loads( text )
    return jsonData

# Start working to get PDO

In [20]:
# Get the team information, which gives us the Team ID which we'll need 
#

jsonResponse = downloadJson('https://statsapi.web.nhl.com/api/v1/teams')

ids = []
for t in jsonResponse['teams']:
    ids.append(t['id'])


In [19]:
team_link_url = 'https://statsapi.web.nhl.com/api/v1/teams/{}/stats'

data = pd.DataFrame(columns=['id', 'Team', 'PDO'])
for id in ids:
    teamJson = downloadJson(team_link_url.format(id))
    theStats = teamJson['stats'][0]['splits'][0]['stat']

    scoringPct = (float(theStats['goalsPerGame']) / float(theStats['shotsPerGame'])) * 100
    savePct = ( 1 - (float(theStats['goalsAgainstPerGame']) / float(theStats['shotsAllowed']))) * 100
    
    pdo = scoringPct + savePct
    
    d = {}
    d['id'] = id
    d['Team'] = teamJson['stats'][0]['splits'][0]['team']['name']
    d['PDO'] = pdo
    data = data.append(d, ignore_index=True)
    
data = data.sort_values(by=['PDO'], ascending=False)
data = data.drop('id', axis=1)
display(data)

,Team,PDO
16,Nashville Predators,102.210187
12,Tampa Bay Lightning,102.099546
28,Winnipeg Jets,102.062452
9,Toronto Maple Leafs,101.942612
19,Colorado Avalanche,101.844662
13,Washington Capitals,101.654077
22,Anaheim Ducks,101.637308
24,Los Angeles Kings,101.354423
27,Minnesota Wild,101.182687
5,Boston Bruins,101.082875
